# Appendix VI: Backward Selection

In [1]:
"""Imports necessary packages"""

import itertools
import math
from typing import Dict, Iterable, List, Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pylab
import scipy
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

sns.set_style("whitegrid")

In [2]:
def backward_p_vals(model_str: str, data: Iterable) -> Dict[str, float]:
    """Obtains the p-values of variables in a model.

    Args:
        model_str (str): the model string as required by statsmodels.api.formulas.ols
        data (Iterable): the data used to fit the model.

    Returns:
        Dict[str, float]: the dictionary of variables of the model (keys) and their associated p-values (values).
    """
    splits = model_str.split(" ~ ")
    expl_vars = splits[-1].split(" + ")
    expl_vars = [var.replace("*", ":") for var in expl_vars]
    model = sm.formula.ols(formula=model_str, data=data)
    model_fitted = model.fit()
    p_vals = model_fitted.pvalues.to_dict()
    p_vals_to_return = {}
    for p_key, p_val in p_vals.items():
        if p_key in expl_vars:
            p_vals_to_return[p_key] = p_val
        for var in expl_vars:
            if var+"[" in p_key:
                p_vals_to_return[var] = p_val
    return p_vals_to_return

In [3]:
def print_p_vals_from_dict(model_dict: Dict[str, float]) -> None:
    """Prints p-values from a dictionary.

    Args:
        model_dict (Dict[str, float]): the dictionary of variables of the model (keys) and their associated p-values (values).
    """
    for var, value in model_dict.items():
        print("p-value of %s: %.8f" % (var, value))

In [4]:
def get_new_backward_model(current_model: str, var_to_remove: str) -> str:
    """Obtains the new model by removing the `var_to_remove` from the `current_model` string.

    Args:
        current_model (str): the current model string as required by statsmodels.api.formulas.ols.
        var_to_remove (str): the variable to remove.

    Returns:
        str: the new model string as required by statsmodels.api.formulas.ols
    """
    var_to_remove = var_to_remove.replace(":", "*")
    splits = current_model.split(" ~ ")    
    expl_vars = splits[-1].split(" + ")
    expl_vars = [var for var in expl_vars if var != var_to_remove]
    return "%s ~ %s" % (splits[0], " + ".join(expl_vars))

In [5]:
def backward_selection(initial_model: str, data: Iterable) -> str: 
    """Performs the backwards selection.

    Args:
        initial_model (str): The initial model string as required by statsmodels.api.formulas.ols.model.
        data (Iterable): the data used to fit the models.

    Returns:
        str: the best model string as required by statsmodels.api.formulas.ols.
    """
    i = 1
    current_model_str = initial_model
    while not current_model_str.endswith(" ~ "):
        print("--- STEP %i ---" % i)
        print("current model: %s" % current_model_str)
        p_vals = backward_p_vals(current_model_str, data)
        print_p_vals_from_dict(p_vals)
        max_key = max(p_vals, key=p_vals.get)
        max_val = p_vals[max_key]
        if max_val > 0.05:
            print("The maximal p-value is of %s" % max_key)
            i += 1
            current_model_str = get_new_backward_model(current_model_str, max_key.split("[")[0])
            
        else:
            print("The maximal p-value is statistically significant.")
            return current_model_str
            
    return current_model_str

In [6]:
data = pd.read_csv("D:/School/frequentist-statistics/ITM-song-popularity/database/itm_songs_preprocessed.csv")
data = data.drop("Unnamed: 0", axis=1)

In [7]:
explanatory_vars = ["name_len", "track_number", "duration", "acousticness", "danceability", "energy", "loudness", "speechiness", "valence", "tempo", "complexity", "age_days", "mode"]

In [8]:
model_string = "popularity_abs ~ %s" % " + ".join(explanatory_vars)
backward_best_abs = backward_selection(model_string, data)
print("The best model for absolute popularity excluding correlations obtained via the backward selection is `%s`." % backward_best_abs)

--- STEP 1 ---
current model: popularity_abs ~ name_len + track_number + duration + acousticness + danceability + energy + loudness + speechiness + valence + tempo + complexity + age_days + mode
p-value of mode: 0.35223609
p-value of name_len: 0.14958524
p-value of track_number: 0.00011500
p-value of duration: 0.12434777
p-value of acousticness: 0.82766002
p-value of danceability: 0.00535535
p-value of energy: 0.87226718
p-value of loudness: 0.21019295
p-value of speechiness: 0.26005450
p-value of valence: 0.33279373
p-value of tempo: 0.38971566
p-value of complexity: 0.83178724
p-value of age_days: 0.00000000
The maximal p-value is of energy
--- STEP 2 ---
current model: popularity_abs ~ name_len + track_number + duration + acousticness + danceability + loudness + speechiness + valence + tempo + complexity + age_days + mode
p-value of mode: 0.33992165
p-value of name_len: 0.14901659
p-value of track_number: 0.00008886
p-value of duration: 0.12309395
p-value of acousticness: 0.87333415

In [9]:
model_string = "popularity_norm ~ %s" % " + ".join(explanatory_vars)
backward_best_rel = backward_selection(model_string, data)
print("The best model for normalised popularity excluding correlations obtained via the backward selection is `%s`." % backward_best_rel)

--- STEP 1 ---
current model: popularity_norm ~ name_len + track_number + duration + acousticness + danceability + energy + loudness + speechiness + valence + tempo + complexity + age_days + mode
p-value of mode: 0.35223609
p-value of name_len: 0.14958524
p-value of track_number: 0.00011500
p-value of duration: 0.12434777
p-value of acousticness: 0.82766002
p-value of danceability: 0.00535535
p-value of energy: 0.87226718
p-value of loudness: 0.21019295
p-value of speechiness: 0.26005450
p-value of valence: 0.33279373
p-value of tempo: 0.38971566
p-value of complexity: 0.83178724
p-value of age_days: 0.00000000
The maximal p-value is of energy
--- STEP 2 ---
current model: popularity_norm ~ name_len + track_number + duration + acousticness + danceability + loudness + speechiness + valence + tempo + complexity + age_days + mode
p-value of mode: 0.33992165
p-value of name_len: 0.14901659
p-value of track_number: 0.00008886
p-value of duration: 0.12309395
p-value of acousticness: 0.873334

In [10]:
correlations = ["duration*complexity", "acousticness*energy", "energy*loudness", "track_number*complexity", "track_number*duration", "duration*loudness", "duration*speechiness", "acousticness*loudness", "danceability*valence", "danceability*complexity", "loudness*complexity", "valence*complexity"]
explanatory_vars.extend(correlations)

In [11]:
model_string = "popularity_abs ~ %s" % " + ".join(explanatory_vars)
best_corr_bs_abs = backward_selection(model_string, data)
print("The best model for absolute popularity including correlations obtained via backward selection is `%s`." % best_corr_bs_abs)

--- STEP 1 ---
current model: popularity_abs ~ name_len + track_number + duration + acousticness + danceability + energy + loudness + speechiness + valence + tempo + complexity + age_days + mode + duration*complexity + acousticness*energy + energy*loudness + track_number*complexity + track_number*duration + duration*loudness + duration*speechiness + acousticness*loudness + danceability*valence + danceability*complexity + loudness*complexity + valence*complexity
p-value of mode: 0.26547739
p-value of name_len: 0.60641621
p-value of track_number: 0.56951738
p-value of duration: 0.66954279
p-value of acousticness: 0.64949962
p-value of danceability: 0.23062817
p-value of energy: 0.64158889
p-value of loudness: 0.62797526
p-value of speechiness: 0.60388923
p-value of valence: 0.31553780
p-value of tempo: 0.47190290
p-value of complexity: 0.71701109
p-value of age_days: 0.00000001
p-value of duration:complexity: 0.25015697
p-value of acousticness:energy: 0.80797109
p-value of energy:loudnes

In [12]:
model_string = "popularity_norm ~ %s" % " + ".join(explanatory_vars)
best_corr_bs_rel = backward_selection(model_string, data)
print("The best model for relative popularity including correlations obtained via backward selection is `%s`." % best_corr_bs_rel)

--- STEP 1 ---
current model: popularity_norm ~ name_len + track_number + duration + acousticness + danceability + energy + loudness + speechiness + valence + tempo + complexity + age_days + mode + duration*complexity + acousticness*energy + energy*loudness + track_number*complexity + track_number*duration + duration*loudness + duration*speechiness + acousticness*loudness + danceability*valence + danceability*complexity + loudness*complexity + valence*complexity
p-value of mode: 0.26547739
p-value of name_len: 0.60641621
p-value of track_number: 0.56951738
p-value of duration: 0.66954279
p-value of acousticness: 0.64949962
p-value of danceability: 0.23062817
p-value of energy: 0.64158889
p-value of loudness: 0.62797526
p-value of speechiness: 0.60388923
p-value of valence: 0.31553780
p-value of tempo: 0.47190290
p-value of complexity: 0.71701109
p-value of age_days: 0.00000001
p-value of duration:complexity: 0.25015697
p-value of acousticness:energy: 0.80797109
p-value of energy:loudne